In [1]:
import numpy as np
import scipy as sp
import scipy.optimize
import teneva
np.random.seed(42)

from numba import jit

In [2]:
@jit
def _poly_arr(x, n, a=-1, b=1):

    x = -1. + 2.*(x - a)/(b-a)
        
    res = np.ones((len(x), n))
    if n == 1:
        return res
    res[:, 1] = x
    for i in range(2, n):
        res[:, i] = 2*x*res[:, i-1] - res[:, i-2]
        
    return res
        
def poly_arr(x, n, a=-1, b=1):
    x = np.asarray(x)
    reduce_dim = x.ndim == 0
    if reduce_dim:
        x = x[None]
        
    res = _poly_arr(x, n, a=a, b=b)
    if reduce_dim:
        res = res[0]
        
    return res

In [3]:
a = -1.
b = +1.
d = 4
n = 50
m = 1.E+5
r = 4
q = 8

In [4]:
I_trn = teneva.sample_lhs([n]*d, m)
I_tst = teneva.sample_lhs([n]*d, m)

X_trn = teneva.ind_to_poi(I_trn, a, b, n, 'cheb')
X_tst = teneva.ind_to_poi(I_tst, a, b, n, 'cheb')

Y_trn = sp.optimize.rosen(X_trn.T)
Y_tst = sp.optimize.rosen(X_tst.T)

In [5]:
Y0 = teneva.tensor_rand([q]*d, r)
teneva.show(Y0)

TT-tensor     4D : |8| |8| |8| |8|
<rank>  =    4.0 :   \4/ \4/ \4/


In [9]:
f = lambda x: poly_arr(x, q, a, b)
A = teneva.als_spectral(X_trn, Y_trn, Y0, f, log=True, nswp=10)

# pre | time:      0.366 | rank:   4.0 | 
#   1 | time:      4.128 | rank:   4.0 | eps: 1.4e+00 | 
#   2 | time:      7.708 | rank:   4.0 | eps: 1.1e-01 | 
#   3 | time:     11.399 | rank:   4.0 | eps: 2.2e-04 | 
#   4 | time:     15.485 | rank:   4.0 | eps: 7.4e-07 | 
#   5 | time:     19.611 | rank:   4.0 | eps: 2.1e-08 | 
#   6 | time:     23.744 | rank:   4.0 | eps: 0.0e+00 | stop: e | 


In [10]:
Y_trn_appr = teneva.cheb_get(X_trn, A, a, b)

np.linalg.norm(Y_trn_appr - Y_trn) / np.linalg.norm(Y_trn)

1.981079307000822e-14

In [11]:
Y_tst_appr = teneva.cheb_get(X_tst, A, a, b)

np.linalg.norm(Y_tst_appr - Y_tst) / np.linalg.norm(Y_tst)

1.9871840843025756e-14

In [ ]:
def cheb_pol(X, a=-1., b=+1., m=10):
    """Compute the Chebyshev polynomials in the given points.

    Args:
        X (np.ndarray): spatial points of interest (it is 2D array of the shape
            [samples, d], where "d" is a number of dimensions).
        a (float, list, np.ndarray): grid lower bounds for each dimension (list
            or np.ndarray of length "d"). It may be also float, then the lower
            bounds for each dimension will be the same.
        b (float, list, np.ndarray): grid upper bounds for each dimension (list
            or np.ndarray of length "d"). It may be also float, then the upper
            bounds for each dimension will be the same.
        m (int): maximum order for Chebyshev polynomial (>= 1). The first "m"
            polynomials (of the order 0, 1, ..., m-1) will be computed.

    Returns:
        np.ndarray: values of the Chebyshev polynomials of the order 0,1,...,m-1
        in X points (it is 3D array of the shape [m, samples, d]).

    Note:
        Before calculating polynomials, the points are scaled from [a, b] to
        standard [-1, 1] limits.

    """
    d = X.shape[-1]
    reps = X.shape[0] if len(X.shape) > 1 else None
    a, b, _ = teneva.grid_prep_opts(a, b, None, d, reps)
    X = (2. * X - b - a) / (b - a)

    T = np.ones([m] + list(X.shape))

    if m < 2:
        return T

    T[1, ] = X.copy()
    for k in range(2, m):
        T[k, ] = 2. * X * T[k - 1, ] - T[k - 2, ]

    return T

In [ ]:
def poly_arr(x, n, a=-1, b=1):
    x = np.asarray(x)
    reduce_dim = x.ndim == 0
    if reduce_dim:
        x = x[None]
        
    res = _poly_arr(x, n, a=a, b=b)
    if reduce_dim:
        res = res[0]
        
    return res

In [12]:
X = np.array([2, 3, 4])

In [13]:
t1 = teneva.cheb_pol(X, a, b, q)

In [14]:
t2 = poly_arr(X, q, a, b)

In [17]:
t1.shape

(8, 3)

In [18]:
t2.shape

(3, 8)